In [21]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count
from pyspark.ml.feature import Imputer, StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline


In [22]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Credit Score Preprocessing") \
    .getOrCreate()


In [23]:
# Load dataset
df = spark.read.csv("../archive/train.csv", header=True, inferSchema=True)
print("Schema:")
df.printSchema()
print("Sample data:")
df.show(100)


Schema:
root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullabl

In [24]:
from pyspark.sql.functions import regexp_replace, when, col

# Define which string‐columns should be numbers
int_cols = [
    "Age", "Num_Bank_Accounts", "Num_Credit_Card",
    "Num_of_Loan", "Delay_from_due_date", "Num_of_Delayed_Payment",
    "Num_Credit_Inquiries"
]
float_cols = [
    "Annual_Income", "Monthly_Inhand_Salary", "Interest_Rate",
    "Changed_Credit_Limit", "Outstanding_Debt", "Credit_Utilization_Ratio",
    "Total_EMI_per_month", "Amount_invested_monthly", "Monthly_Balance"
]

# For each of these, strip out any non-digit/dot characters,
# and turn "NA" or empty strings into null
for c in int_cols + float_cols:
    df = df.withColumn(
        c,
        when(
            col(c).isNull() | (col(c) == "NA") | (col(c) == ""),
            None
        ).otherwise(
            regexp_replace(col(c), "[^0-9\\.]", "")
        )
    )


In [25]:
from pyspark.sql.types import IntegerType, DoubleType

# Cast integer columns
for c in int_cols:
    df = df.withColumn(c, col(c).cast(IntegerType()))

# Cast float columns
for c in float_cols:
    df = df.withColumn(c, col(c).cast(DoubleType()))

# Quick check
df.printSchema()
df.show(5)


root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: double (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: double (nullable = true)
 |-- Num_of_Loan: integer (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: integer (nullable = true)
 |-- Changed_Credit_Limit: double (nullable = true)
 |-- Num_Credit_Inquiries: integer (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: double (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullable = t

In [26]:
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import DoubleType

# 1) Extract the “Years” and “Months” as strings, then cast
df = (
    df
    .withColumn(
        "Credit_History_Years",
        regexp_extract(col("Credit_History_Age"), r"(\d+)\s+Years", 1).cast(DoubleType())
    )
    .withColumn(
        "Credit_History_Months",
        regexp_extract(col("Credit_History_Age"), r"(\d+)\s+Months", 1).cast(DoubleType())
    )
)

# 2) Combine into a single fractional years column
df = df.withColumn(
    "Credit_History_Age_Years",
    col("Credit_History_Years") + (col("Credit_History_Months") / 12)
)

# 3) Inspect the result
df.select(
    "Credit_History_Age",
    "Credit_History_Years",
    "Credit_History_Months",
    "Credit_History_Age_Years"
).show(10, truncate=False)

# Drop the old string column and intermediate year/month columns, then rename the computed column
df = (
    df
    .drop("Credit_History_Age", "Credit_History_Years", "Credit_History_Months")
    .withColumnRenamed("Credit_History_Age_Years", "Credit_History_Age")
)

# Verify the replacement
df.select("Credit_History_Age").show(5, truncate=False)


+---------------------+--------------------+---------------------+------------------------+
|Credit_History_Age   |Credit_History_Years|Credit_History_Months|Credit_History_Age_Years|
+---------------------+--------------------+---------------------+------------------------+
|22 Years and 1 Months|22.0                |1.0                  |22.083333333333332      |
|NA                   |NULL                |NULL                 |NULL                    |
|22 Years and 3 Months|22.0                |3.0                  |22.25                   |
|22 Years and 4 Months|22.0                |4.0                  |22.333333333333332      |
|22 Years and 5 Months|22.0                |5.0                  |22.416666666666668      |
|22 Years and 6 Months|22.0                |6.0                  |22.5                    |
|22 Years and 7 Months|22.0                |7.0                  |22.583333333333332      |
|NA                   |NULL                |NULL                 |NULL          

In [27]:
# Summarize missing values for each column
missing_summary = df.select([
    count(when(col(c).isNull() | isnan(c), c)).alias(c)
    for c in df.columns
])
missing_summary.show(truncate=False)


+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+---------------------+-------------------+-----------------------+-----------------+---------------+------------+------------------+
|ID |Customer_ID|Month|Name|Age|SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behaviour|Monthly_Balance|Credit_Score|Credit_History_Age|
+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+---

In [28]:
# Drop columns with >50% missing values
total_count = df.count()
print("Total count:", total_count)
threshold = total_count * 0.5
to_drop = [
    c for c in missing_summary.columns
    if missing_summary.collect()[0][c] > threshold
]
print("Dropping columns:", to_drop)
df = df.drop(*to_drop)


Total count: 100000
Dropping columns: []


In [29]:
#Drop irrelevant columns
#rethink occupation
irrelevant_columns = ['ID', 'Customer_ID', 'Name' ,'Age', 'SSN','Month','Occupation']
df = df.drop(*irrelevant_columns)
print("Columns after dropping irrelevant ones:", df.columns)


Columns after dropping irrelevant ones: ['Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score', 'Credit_History_Age']


In [30]:
# Impute numeric columns with median
numeric_cols = [c for c, t in df.dtypes if t in ("double", "int", "float")]
imputer = Imputer(
    strategy="median",
    inputCols=numeric_cols,
    outputCols=[f"{c}_imp" for c in numeric_cols]
)
df = imputer.fit(df).transform(df)

# Drop original numeric cols and rename _imp back to original names
for c in numeric_cols:
    df = df.drop(c).withColumnRenamed(f"{c}_imp", c)
# Check the data type of the column 'Num_of_Delayed_Payment'
column_type = dict(df.dtypes).get('Num_of_Delayed_Payment', None)
print(f"Data type of 'Num_of_Delayed_Payment': {column_type}")
df.show(100)


Data type of 'Num_of_Delayed_Payment': int
+--------------------+----------+---------------------+--------------------+------------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+-------------------+-----------------------+------------------+------------------+
|        Type_of_Loan|Credit_Mix|Payment_of_Min_Amount|   Payment_Behaviour|Credit_Score|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Total_EMI_per_month|Amount_invested_monthly|   Monthly_Balance|Credit_History_Age|
+--------------------+----------+---------------------+--------------------+------------+-------------+---------------------+-----------------+---------------+--------

In [31]:
df.describe().show()

+-------+--------------------+----------+---------------------+--------------------+------------+------------------+---------------------+------------------+-----------------+------------------+-----------------+-------------------+----------------------+--------------------+--------------------+------------------+------------------------+-------------------+-----------------------+--------------------+-------------------+
|summary|        Type_of_Loan|Credit_Mix|Payment_of_Min_Amount|   Payment_Behaviour|Credit_Score|     Annual_Income|Monthly_Inhand_Salary| Num_Bank_Accounts|  Num_Credit_Card|     Interest_Rate|      Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|  Outstanding_Debt|Credit_Utilization_Ratio|Total_EMI_per_month|Amount_invested_monthly|     Monthly_Balance| Credit_History_Age|
+-------+--------------------+----------+---------------------+--------------------+------------+------------------+---------------------+--------

In [32]:
# Fill missing categorical columns with "Missing"
categorical_cols = [c for c, t in df.dtypes if t == "string"]
df = df.na.fill({c: "Missing" for c in categorical_cols})


In [33]:
# StringIndexer for each categorical column
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in categorical_cols
]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)
df.show(1000)

#drop original categorical columns
for c in categorical_cols:
    df = df.drop(c)


+--------------------+----------+---------------------+--------------------+------------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+-------------------+-----------------------+------------------+------------------+----------------+--------------+-------------------------+---------------------+----------------+
|        Type_of_Loan|Credit_Mix|Payment_of_Min_Amount|   Payment_Behaviour|Credit_Score|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Total_EMI_per_month|Amount_invested_monthly|   Monthly_Balance|Credit_History_Age|Type_of_Loan_idx|Credit_Mix_idx|Payment_of_Min_Amount_idx|Payment_Behaviour_idx|Credit_Score_idx|
+---------------

In [34]:
# OneHotEncoder for indexed columns
encoder = OneHotEncoder(
    inputCols=[f"{c}_idx" for c in categorical_cols],
    outputCols=[f"{c}_ohe" for c in categorical_cols]
)
df = encoder.fit(df).transform(df)



In [35]:
# Assemble numeric + one-hot features into a single vector
feature_cols = numeric_cols + [f"{c}_ohe" for c in categorical_cols]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="raw_features")
df = assembler.transform(df)


In [36]:
# Rename raw_features to features for tree-based algorithms
# (RandomForest, GBT do not require scaled inputs)
df_approachA = df.withColumnRenamed("raw_features", "features")

# You can now feed df_approachA into your RandomForestClassifier or GBTClassifier directly:
# e.g. rf = RandomForestClassifier(featuresCol="features", labelCol="Credit Score", ...)

print(df.dtypes)

[('Annual_Income', 'double'), ('Monthly_Inhand_Salary', 'double'), ('Num_Bank_Accounts', 'int'), ('Num_Credit_Card', 'int'), ('Interest_Rate', 'double'), ('Num_of_Loan', 'int'), ('Delay_from_due_date', 'int'), ('Num_of_Delayed_Payment', 'int'), ('Changed_Credit_Limit', 'double'), ('Num_Credit_Inquiries', 'int'), ('Outstanding_Debt', 'double'), ('Credit_Utilization_Ratio', 'double'), ('Total_EMI_per_month', 'double'), ('Amount_invested_monthly', 'double'), ('Monthly_Balance', 'double'), ('Credit_History_Age', 'double'), ('Type_of_Loan_idx', 'double'), ('Credit_Mix_idx', 'double'), ('Payment_of_Min_Amount_idx', 'double'), ('Payment_Behaviour_idx', 'double'), ('Credit_Score_idx', 'double'), ('Type_of_Loan_ohe', 'vector'), ('Credit_Mix_ohe', 'vector'), ('Payment_of_Min_Amount_ohe', 'vector'), ('Payment_Behaviour_ohe', 'vector'), ('Credit_Score_ohe', 'vector'), ('raw_features', 'vector')]


In [37]:
# from pyspark.ml.feature import VectorAssembler, StandardScaler

# # 1) Assemble just the numeric columns into "raw_num"
# num_assembler = VectorAssembler(inputCols=numeric_cols, outputCol="raw_num")
# df2 = num_assembler.transform(df)

# # 2) Scale numeric features WITHOUT centering to preserve sparsity of one-hot
# scaler = StandardScaler(
#     inputCol="raw_num", 
#     outputCol="scaled_num", 
#     withMean=False,   # do not center
#     withStd=True      # scale to unit variance
# )
# df2 = scaler.fit(df2).transform(df2)

# # 3) Re-assemble scaled numeric + all one-hot columns into final "features"
# final_assembler = VectorAssembler(
#     inputCols=["scaled_num"] + [f"{c}_ohe" for c in categorical_cols],
#     outputCol="features"
# )
# df_approachB = final_assembler.transform(df2)

# # Now df_approachB.features is ready for LogisticRegression, LinearSVC, etc.


In [38]:
# normalize fetaures

from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=2.0)
df = normalizer.transform(df_approachA)



In [39]:

# Save the preprocessed data to CSV files
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
train_df.show(5)

+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+-------------------+-----------------------+------------------+------------------+----------------+--------------+-------------------------+---------------------+----------------+------------------+--------------+-------------------------+---------------------+----------------+--------------------+--------------------+
|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Total_EMI_per_month|Amount_invested_monthly|   Monthly_Balance|Credit_History_Age|Type_of_Loan_idx|Credit_Mix_idx|Payment_of_Min_Amount_idx|Payment_Behaviour_idx|Credit_Score_idx|  Type_of_Loan_ohe|Credit_Mix_ohe|Payment_of_Min_Amo

In [41]:
# When you’re done, you can stop Spark
# spark.stop()

#cast credit_Score to int
train_df = train_df.withColumn("Credit_Score_idx", col("Credit_Score_idx").cast(IntegerType()))

In [42]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="norm_features",
    labelCol="Credit_Score_idx",
    maxIter=50,
    regParam=0.1,
    elasticNetParam=0.0  # L2 regularization
)
lrModel = lr.fit(train_df)


In [ ]:
# Re-initialize Spark, reload and preprocess data, split train_df/test_df, and fit lrModel here…
# Then paste the evaluation cell:
predictions = lrModel.transform(test_df)
predictions.select("Credit_Score_idx", "prediction", "probability").show(10)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="Credit_Score_idx", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
# ... similarly for precision, recall, f1 ...
print(f"Accuracy = {accuracy:.3f}")
# etc.


+----------------+----------+--------------------+
|Credit_Score_idx|prediction|         probability|
+----------------+----------+--------------------+
|             1.0|       1.0|[8.30565115783930...|
|             0.0|       0.0|[0.99044877630206...|
|             0.0|       0.0|[0.98112116210343...|
|             1.0|       1.0|[0.00255792925407...|
|             1.0|       1.0|[0.00348172090611...|
|             1.0|       1.0|[0.00385700472484...|
|             1.0|       1.0|[0.00155225262204...|
|             1.0|       1.0|[7.42126347050647...|
|             1.0|       1.0|[0.00188289246280...|
|             1.0|       1.0|[0.00528936971599...|
+----------------+----------+--------------------+
only showing top 10 rows

Accuracy = 0.914


: 